# Investigating the Accuracy of the Hartree-Fock Energy of H2 by Comparing it with the Second-Order Moller-Plesset Perturbation Theory Energy

### Calculating the Second-Order Moller-Plesset Perturbation Theory (MP2) energy of the H2 molecule in the reference state (1100)

In [59]:
def hydrogen_reference_state_mp2_energy():
    """
    Calculates the Second-Order Moller-Plesset Perturbation Theory (MP2) energy of the H2 molecule in the reference state (1100).
    The reference energies and wave functions are obtained using psi4. From this, orbital occupation and energy are obtained.
    The Electron Repulsion Integral (ERI) is obtained using psi4.
    This is in the Atomic Orbital (AO) basis and needs to be transformed to the Molecular Orbital (MO) basis before it can be used.
    This is done by obtaining the orbital coefficient matrix, C, from the reference wavefunction.
    ERI in the MO basis along with the energy denomiators of the orbitals are used to calculate the total mp2 energy
    """
    # Setting computational resources and output options
    psi4.set_memory(int(2e9))
    psi4.core.set_output_file('output.dat', False)
    computational_options = {
        'basis': '6-31g',
        'scf_type': 'pk',
        'mp2_type': 'conv',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8
    }
    psi4.set_options(computational_options)

    # Defining the H-2 molecule
    hydrogen_molecule = psi4.geometry("""
        H 0 0 -0.6614
        H 0 0 0.6614 
        symmetry c1
    """)

    # Doing the SCF calculation to obtain reference the wavefunction and energy
    ref_energy, ref_wavefunction = psi4.energy('scf', return_wfn=True)

    # Getting the number of doubly occupied orbitals and total number of orbitals
    num_doubly_occupied = ref_wavefunction.nalpha()
    num_molecular_orbitals = ref_wavefunction.nmo()

    # Getting orbital energies
    orbital_energies = np.asarray(ref_wavefunction.epsilon_a())
    occupied_orbital_energies = orbital_energies[:num_doubly_occupied]
    virtual_orbital_energies = orbital_energies[num_doubly_occupied:]

    # Making the electron repulsion integral (ERI) tensor
    molecular_integrals = psi4.core.MintsHelper(ref_wavefunction.basisset())
    eri_tensor = np.asarray(molecular_integrals.ao_eri())

    # Obtaining molecular orbital (MO) coefficients from the SCF wavefunction
    mo_coefficients = np.asarray(ref_wavefunction.Ca())
    occupied_mo_coefficients = mo_coefficients[:, :num_doubly_occupied]
    virtual_mo_coefficients = mo_coefficients[:, num_doubly_occupied:]

    # Transforming the ERI tensor to the MO basis
    intermediate_tensor = np.einsum('pi,pqrs->iqrs', occupied_mo_coefficients, eri_tensor, optimize=True)
    intermediate_tensor = np.einsum('qa,iqrs->iars', virtual_mo_coefficients, intermediate_tensor, optimize=True)
    intermediate_tensor = np.einsum('iars,rj->iajs', intermediate_tensor, occupied_mo_coefficients, optimize=True)
    eri_mo_basis = np.einsum('iajs,sb->iajb', intermediate_tensor, virtual_mo_coefficients, optimize=True)

    # Calculating the MP2 energy- sum of the interactions of particles with the same spin and the interactions of particles with different spins
    energy_denominator = 1 / (occupied_orbital_energies.reshape(-1, 1, 1, 1) - virtual_orbital_energies.reshape(-1, 1, 1) +
                              occupied_orbital_energies.reshape(-1, 1) - virtual_orbital_energies)
    mp2_diff_spins = np.einsum('iajb,iajb,iajb->', eri_mo_basis, eri_mo_basis, energy_denominator, optimize=True)
    mp2_same_spins = np.einsum('iajb,iajb,iajb->', eri_mo_basis, eri_mo_basis - eri_mo_basis.swapaxes(1,3), energy_denominator, optimize=True)
    total_mp2_energy = ref_energy + mp2_diff_spins + mp2_same_spins

    return total_mp2_energy.item()
energy_mp2 = hydrogen_reference_state_mp2_energy()

-1.0570227140108972


### Calculating the Hartree-Fock (HF) energy of the H2 molecule in the reference state (1100)

In [60]:
symbols  = ['H', 'H']
geometry = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0]], requires_grad = False)
alpha = np.array([[3.42525091, 0.62391373, 0.1688554 ],
                   [3.42525091, 0.62391373, 0.1688554]], requires_grad=True)
mol = qml.qchem.Molecule(symbols, geometry, alpha=alpha)
args = [alpha]

energy_hf = np.array(qml.qchem.hf_energy(mol)(*args)).item()

### Comparison between Hartree-Fock energy and Second-Order Moller-Plesset Perturbation Theory Energy for H2 in the reference state 1100

In [67]:
print('Second-Order Moller-Plesset Perturbation Theory Energy:', energy_mp2)
print('Hartree-Fock energy:', energy_hf)
print(f'Accuracy: {100 * np.positive(energy_hf - energy_mp2)/ energy_mp2:.2f}%')

Second-Order Moller-Plesset Perturbation Theory Energy: -1.0570227140108972
Hartree-Fock energy: -1.0659994584784829
Accuracy: 0.85%


#### We can see that whilst Second-Order Moller-Plesset Perturbation Theory gives a more accurate result by accounting for electron repulsion, in the H2 molecule, the result of Hartree-Fock is very close and can be considered a good approximation